In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
import pathlib
import pandas as pd 
train_csv = pd.read_csv('/kaggle/input/landmark-recognition-2020/train.csv')
train_csv.head()

In [ ]:
num_classes = len(train_csv['landmark_id'].unique())
print(num_classes)

In [ ]:
train_csv = pd.read_csv('/kaggle/input/landmark-recognition-2020/train.csv')
train_csv.head(10)

# put .jpg into the file name
def add_txt(fn):
    return fn+'.jpg'

train_csv['id'] = train_csv['id'].apply(add_txt)



# choose those labels with more than 200 images, and choose the first 200 images of each label
# move every training files to the same folder
%cd /kaggle/working
if not os.path.exists('training'):
    os.mkdir('training')
if not os.path.exists('validation'):
    os.mkdir('validation')
if not os.path.exists('testing'):
    os.mkdir('testing')    

import shutil
import random

label_list = train_csv['landmark_id'].unique()
cnt = 0
final_label_list = []

for label in list(label_list): # label order by random
    file_list = list(train_csv['id'][train_csv['landmark_id']==label])
    if len(file_list) >= 200:
        final_label_list.append(label)
        if not os.path.exists('/kaggle/working/training/'+str(label)):
            os.mkdir('/kaggle/working/training/'+str(label))
        if not os.path.exists('/kaggle/working/validation/'+str(label)):
            os.mkdir('/kaggle/working/validation/'+str(label))
        if not os.path.exists('/kaggle/working/testing/'+str(label)):
            os.mkdir('/kaggle/working/testing/'+str(label))
        for file in file_list[:120]:  # 120 files for training
            src = '/kaggle/input/landmark-recognition-2020/train/'+file[0]+'/'+file[1]+'/'+file[2]+'/'+file
            dst = '/kaggle/working/training/'+str(label)+'/'+file
            if not os.path.exists(dst):
                shutil.copyfile(src, dst)
        for file in file_list[120:160]: # 40 files for validation
            src = '/kaggle/input/landmark-recognition-2020/train/'+file[0]+'/'+file[1]+'/'+file[2]+'/'+file
            dst = '/kaggle/working/validation/'+str(label)+'/'+file
            if not os.path.exists(dst):
                shutil.copyfile(src, dst)
        for file in file_list[160:200]: # 40 files for testing
            src = '/kaggle/input/landmark-recognition-2020/train/'+file[0]+'/'+file[1]+'/'+file[2]+'/'+file
            dst = '/kaggle/working/testing/'+str(label)+'/'+file
            if not os.path.exists(dst):
                shutil.copyfile(src, dst)
        cnt += 1
    if cnt == 100: # only need 100 labels
        break

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = '/kaggle/working/training'
validation_dir = '/kaggle/working/validation'
test_dir = '/kaggle/working/testing'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size = 32,
    class_mode='categorical',
    seed=42)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(256, 256),
    batch_size = 32,
    class_mode='categorical',
    seed=42)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size = 1,
    class_mode='categorical',
    seed=42)

In [ ]:
from tensorflow.keras.applications import ResNet50V2
conv_base = ResNet50V2(include_top=False,
                    weights="imagenet",
                    input_shape=(256, 256, 3))
conv_base.trainable = True
conv_base.summary()

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D, Flatten, Conv2D, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
import tensorflow as tf

model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(Dense(100, activation='softmax'))
model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    verbose=2
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, '#21466C', label='Training acc')
plt.plot(epochs, val_acc, '#ff0051', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, '#21466C', label='Training loss')
plt.plot(epochs, val_loss, '#ff0051', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
scores = model.evaluate(test_generator)
print('loss:', scores[0])
print('accuracy:', scores[1])